In [7]:
from dask.distributed import Client
client = Client()
# client = Client('127.0.0.1:8786')
client

Client Scheduler: tcp://127.0.0.1:59399,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


In [8]:
import os, math
from collections import Counter

size_mbs = lambda x: int(os.path.getsize(x)
                         / math.pow(1024, 2))

def file_sizes(directory): 
    d = "../data/osm/ny/" + directory
    sizes = [size_mbs(os.path.join(d, f)) \
             for f in os.listdir(d)]
    c = Counter(sizes)
    for size, count in c.items():
        print("{:3d} x {:3d} MB".format(count, size))

In [9]:
file_sizes("500mb")

  2 x 476 MB
  1 x 289 MB


In [15]:
import dask.bag as db
from operator import itemgetter

extract_tag = itemgetter(1, 3)

def top_tags(data):
    return data.filter(
        lambda x: "tag" in x
    ).foldby(
        lambda x: extract_tag(x.split('"')),
        lambda x, _: x + 1, 0
    ).topk(10, itemgetter(1))

In [16]:
data = db.read_text("../data/osm/ny/500mb/*")
task = top_tags(data)
task

dask.bag<topk-aggregate, npartitions=1>

In [17]:
task.visualize("task1", format="svg");

In [18]:
task.compute()

[(('tiger:reviewed', 'no'), 61282),
 (('tiger:cfcc', 'A41'), 59104),
 (('highway', 'residential'), 50597),
 (('created_by', 'ArcGIS Exporter'), 22285),
 (('nycdoitt:bin', '3000000'), 18263),
 (('tiger:name_type', 'St'), 17259),
 (('tiger:name_type', 'Ave'), 16343),
 (('nycdoitt:bin', '4000000'), 16301),
 (('highway', 'traffic_signals'), 15358),
 (('height', '7.8'), 12935)]

In [19]:
file_sizes("5mb")
data = db.read_text("../data/osm/ny/5mb/*")
task = top_tags(data)
task

260 x   4 MB
  1 x   2 MB


dask.bag<topk-aggregate, npartitions=1>

In [20]:
task.compute()

[(('highway', 'traffic_signals'), 9039),
 (('created_by', 'ArcGIS Exporter'), 8002),
 (('tiger:county', 'Nassau, NY'), 4890),
 (('amenity', 'bicycle_parking'), 4539),
 (('tiger:county', 'Essex, NJ'), 4018),
 (('cityracks.large', '0'), 3479),
 (('tiger:county', 'Richmond, NY'), 3417),
 (('tiger:county', 'Queens, NY'), 3232),
 (('cityracks.small', '1'), 3156),
 (('capacity', '2'), 3057)]

In [21]:
file_sizes("100mb")
data = db.read_text("../data/osm/ny/100mb/*")
task = top_tags(data)
task

 13 x  95 MB
  1 x   2 MB


dask.bag<topk-aggregate, npartitions=1>

In [22]:
task.compute()

[(('tiger:reviewed', 'no'), 39799),
 (('tiger:cfcc', 'A41'), 39376),
 (('highway', 'residential'), 37131),
 (('created_by', 'ArcGIS Exporter'), 22285),
 (('highway', 'traffic_signals'), 15141),
 (('tiger:name_type', 'St'), 11688),
 (('tiger:name_type', 'Ave'), 9743),
 (('tiger:county', 'Bergen, NJ'), 7048),
 (('tiger:county', 'Essex, NJ'), 6206),
 (('tiger:county', 'Queens, NY'), 5827)]

In [23]:
data = db.read_text("../data/osm/southamerica/southam*")
task = top_tags(data)
task

dask.bag<topk-aggregate, npartitions=1>

In [24]:
task.compute()

[(('created_by', 'JOSM'), 364794),
 (('source', 'ICRC, DigitalGlobe 120311'), 12258),
 (('source_ref', 'http://earth-info.nga.mil/gns/html/cntyfile/ar.zip'), 5581),
 (('source', 'ISS-004-E-12987, 12988, 12989 Imagery'), 4871),
 (('gns_classification', 'PPL'), 4128),
 (('source', 'Lakewalker plugin'), 2275),
 (('highway', 'traffic_signals'), 2033),
 (('waterway', 'river'), 1754),
 (('gns_classification', 'STM'), 1727),
 (('place', 'isolated_dwelling'), 1662)]